# Implémentations récursive vs. itérative :
## Eviter de faire recours au récursif dans l'algorithmique (des graphes)

Dans l'informatique en général et dans l'algorithmique des graphes, la récursivité est souvent utilisée pour résoudre de nombreux problèmes. 

Une fonction récursive, c'est simplement une fonction qui s'appelle elle-même. Cela permet de résoudre des problèmes de manière assez intuitive et simple pour le programmeur.

Un exemple très basique de fonction qu'on peut logiquement implémenter de manière récursive est la fonction factorielle. Une implémentation se trouve dans la cellule ci-dessous.

In [1]:
def recursive_factorial(n):
    return recursive_factorial(n-1)*n if n > 1 else 1

Il est toujours possible de transformer une fonction récursive en une fonction iterative et qui va donc utiliser une boucle while (ou for) au lieu de s'appeler elle-même. Dans le cas de la fonction factorielle, une forme itérative de la procédure est assez simple également:

In [11]:
import time
def iterative_factorial(n):
    total = 1
    while n > 1:
        total *= n
        n -= 1
    return total

# vérifions rapidement que les deux implémentations provoquent bien les mêmes résultats
start = time.time()
assert recursive_factorial(5) == iterative_factorial(5), "test 1: échoué"
end = time.time()
print("test 1: ok", end-start)

start = time.time()
assert recursive_factorial(20) == iterative_factorial(20), "test 2: échoué"
end = time.time()
print("test 2: ok", end-start)

test 1: ok 4.410743713378906e-05
test 2: ok 3.3855438232421875e-05


Comme énoncé plus haut, il est souvent possible d'utiliser des procédures récursives dans l'algorithmique des graphes et vous pourrez trouver de nombreux pseudo-codes ou codes complets sur le web qui suggèrent ce genre d'implémentation. 

Cependant, l'utilisation de fonctions récursives peut poser problèmes dans certains cas. En effet, à chaque fois qu'une fonction s'appelle elle-même, cet appel ainsi que les variables locales qu'il utilise en argument sont enregistrés dans une zone de la mémoire nommée la pile d'exécution (call stack). Cet appel n'en est retiré qu'une fois l'exécution complètement terminée et donc le résultat retourné. Cela signifie qu'une fonction qui s'appelle 10000 fois d'affilée de manière récursive va ajouter 10000 enregistrements de ces appels de fonctions sur la pile. 

La pile ayant une taille assez limitée, elle a tendance à se remplir complètement lorsqu'on rencontre un problème trop gros ou une instance avec une forme particulière qui provoquerait une profondeur excessive de la récursivité. C'est pourquoi il est de bon usage d'opter pour des solutions itératives.

Un exemple très basique de procédure que l'on peut implémenter de manière récursive ou itérative dans l'algorithmique des graphes est la recherche en profondeur (Depth-First Search ou dfs en anglais) dans un graphe dirigé et non-pondéré. Deux implémentations pour ces deux méthodes sont présentées ci-dessous.

In [12]:
class Graph:
    
    def __init__(self, adj):
        self.adj = adj # liste d'adjacence du graphe (adj[i] contient j si il y a une arête du noeud i à j)
        self.dist = [float('inf') for i in range(len(adj))] # liste dans laquelle on enregistre la distance à chaque noeud en partant du noeud donné
    
    def recursive_dfs(self, u, curr_dist = 0):
        """
        u : noeud exploré (noeud de départ pour le premier appel)
        curr_dist : distance actuelle par rapport au noeud de départ (0 pour le premier appel)
        """
        if self.dist[u] > curr_dist:
            self.dist[u] = curr_dist
            for v in self.adj[u]:
                self.recursive_dfs(v, curr_dist + 1) # appel récursif
                
    def iterative_dfs(self, u):
        """
        u : noeud de départ
        """
        q = [] 
        q.append((u, 0)) 
        while q:
            u, curr_dist = q.pop(-1) # on retire le dernier élément et donc le dernier ajouté pour dfs (remplace -1 par 0 pour bfs)
            if self.dist[u] > curr_dist:
                self.dist[u] = curr_dist
                for v in self.adj[u]:
                    q.append((v, curr_dist + 1))
                    
    def reset_dist(self):
        """
        Fonction qui réinitialise la distance calculée à chaque noeud 
        """
        self.dist = [float('inf') for i in range(len(self.adj))]

Ces deux implémentations font exactement les mêmes opérations pour obtenir le même résultat à l'exception qu'une fonction est récursive et l'autre itérative. Cela signifie qu'elles ont exactement la même complexité. Dans la cellule suivante, on met en avant la très légère différence de temps d'exécution entre les deux méthodes pour démontrer un fonctionnement très similaire, voire considérable comme équivalent.

Une autre solution aurait été de compter le nombre d'opérations élémentaires (i.e. nombre de comparaisons '>', '>=', d'allocation '=', opérations '+', '-', ...) que chaque fonction provoque lors d'un appel. Ce nombre d'opération peut être approximé en comptant le nombre de tours de boucle en itératif ou bien le nombre d'appels récursifs.

In [14]:
import random

def generate_random_graph(n_node, n_edge):
    adj = [[] for _ in range(n_node)]
    while n_edge > 0 :
        a, b = random.randint(0, n_node-1), random.randint(0, n_node-1)
        if b not in adj[a] :
            adj[a].append(b)
            n_edge -= 1
    return Graph(adj)

import time
import matplotlib.pyplot as plt
import numpy as np

dt_rec = []
dt_ite = []

for i in range(10):
    G = generate_random_graph(500, 10000)

    t_start = time.time()
    G.recursive_dfs(0)
    t_end = time.time()
    dt_rec.append(t_end - t_start) 
    recursive_dist = G.dist

    G.reset_dist()

    t_start = time.time()
    G.iterative_dfs(0)
    t_end = time.time()
    dt_ite.append(t_end - t_start) 
    iterative_dist = G.dist

    assert recursive_dist == iterative_dist


print(f'Le dfs récursif a duré en moyenne {np.mean(dt_rec):.4} secondes')
print(f'Le dfs iteratif a duré en moyenne {np.mean(dt_ite):.4} secondes')
print(f"La plus grande différence de temps d'exécution observée est {np.max(np.abs(np.array(dt_rec) - np.array(dt_ite))):.4} seconds")

Le dfs récursif a duré en moyenne 0.1588 secondes
Le dfs iteratif a duré en moyenne 0.1702 secondes
La plus grande différence de temps d'exécution observée est 0.01896 seconds


Maintenant que l'on sait que les deux algorithmes donnent bien les mêmes résultats et ont la même complexité, il ne reste plus qu'à montrer pourquoi l'utilisation de la récursivité peut poser problème dans le cas où une récursion trop profonde serait observée. Ceci peut se faire très simplement dans notre cas via l'utilisation de nos fonctions sur des graphes chemins.

In [16]:
def generate_path_graph(n_node):
    adj = [[i+1] if i < n_node-1 else [] for i in range(n_node)]
    return Graph(adj)

for n_node in [10, 100, 1000, 10000, 100000, 1000000]:

    G = generate_path_graph(n_node)
    
    G.iterative_dfs(0)
    iterative_dist = G.dist
    
    G.reset_dist()
    
    try:
        G.recursive_dfs(0)
    except RecursionError:
        print(f"La pile d'exécution a débordé car trop de niveaux de récursion avec le chemin à {n_node} noeuds")
        break
        
    recursive_dist = G.dist
    assert iterative_dist == recursive_dist
    print(f"Aucun souci pour l'algorithme récursif sur le chemin à {n_node} noeuds")

Aucun souci pour l'algorithme récursif sur le chemin à 10 noeuds
Aucun souci pour l'algorithme récursif sur le chemin à 100 noeuds
Aucun souci pour l'algorithme récursif sur le chemin à 1000 noeuds
La pile d'exécution a débordé car trop de niveaux de récursion avec le chemin à 10000 noeuds
